# Arrhythmia detection

In [ ]:
import logging
import os
import pickle

import neurokit2 as nk
import wfdb

## Load the database if it exists

In [ ]:
dbdir = 'mit-bih-arrhythmia-database-1.0.0'
if not os.path.isdir(dbdir):
    logging.error('Please replace dbdir with the db directory')
    raise FileNotFoundError(dbdir)

## Detect P, Q, R, S, T waves

![alt text](ecg.png "Title")

In [10]:
# Read from the docs
sampling_rate = 360
record = {'Sample_Symbol': ([]), 'ECG_Signal': ([]), 'R_Peaks': ({}), 'waves_peaks': ({})}
records = []

# Load records file by file
for _ in range(100, 220):
    try:
        bih_record = wfdb.rdrecord('{}/{}'.format(dbdir, _))
        annotation = wfdb.rdann('{}/{}'.format(dbdir, _), 'atr')
    except FileNotFoundError:
        continue

    # Create a list of tuples of beat symbols and the sample at which it is annotated 
    record['Sample_Symbol'] = (
    [(i / sampling_rate, s) for i, s in zip(annotation.sample[:], annotation.symbol[:])])
    # Read the signal from the bih_record
    ecg_signal = bih_record.p_signal[:, 0]
    record['ECG_Signal'] = ecg_signal
    # Find all r peaks in a record
    _, r_peaks = nk.ecg_peaks(ecg_signal, sampling_rate=sampling_rate)
    record['R_Peaks'] = r_peaks
    # Find P, Q, S, T, P-onset and T-offset
    _, waves_peaks = nk.ecg_delineate(ecg_signal, r_peaks, sampling_rate=sampling_rate,
                                      method="peak")
    record['waves_peaks'] = waves_peaks
    records.append(record)

# Save the data to a file (search of waves takes approximately 17min)
# with open('records.pkl', 'wb') as file:
#     pickle.dump(records, file)

## Map annotations to R waves
(\+)      Rhythm change

(N)       Normal beat

(A)		Atrial premature beat

(/)		Paced beat

(f)		Fusion of paced and normal beat

(~)		Change in signal quality

(V)		Premature ventricular contraction

(x)		Non-conducted P-wave (blocked APC)

(L)		Left bundle branch block beat

(R)		Right bundle branch block beat

(F)		Fusion of ventricular and normal beat

In [3]:
# Load records from the file
with open('records.pkl', 'rb') as file:
    records = pickle.load(file)
    


## Distribution of classes

In [11]:
# labels, values = zip(*Counter([s[0] for s in slices]).items())
# plt.bar(labels, values)
# for i, v in enumerate(values):
#     plt.text(i, v + 0.1, str(v), ha='center', va='bottom')
# plt.title('Distribution of Beat and Non-beat annotations')
# plt.xlabel('Annotation')
# plt.ylabel('Frequency')
# plt.show()

## Features for classification:
* age
* sex
* PR interval
* RT interval

## SVM model training
### SVM Gaussian radial basis function